In [1]:
# Libraries
import pandas as pd

In [2]:
# Import CSV files

data_df = pd.read_csv("csv/purchase_data.csv")


In [3]:
# Data Type Identification

data_df = data_df.set_index("Purchase ID")
#data_df.dtypes

In [4]:
all_players = data_df["Gender"].count()
all_players

780

# Player Count

In [5]:
# Display Total Number of Unique Players

total_players = data_df["SN"].nunique()
total_players_dict = [{"Total Players":total_players}]
total_players_df = pd.DataFrame(total_players_dict)
total_players_df

,Total Players
0,576


# Purchase Analysis (Total)

In [6]:
# Purchasing Analysis (Total)

unique_items = data_df["Item ID"].nunique()
average_price = '${:,.2f}'.format(data_df["Price"].mean())
num_pur = data_df["SN"].count()
tot_rev = '${:,.2f}'.format(data_df["Price"].sum())


analysis_dict = [{"Number of Unique Items":unique_items,
                  "Average Price":average_price,
                  "Number of Purchases":num_pur,
                  "Total Revenue":tot_rev}]
analysis_df = pd.DataFrame(analysis_dict)
analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


# Gender Demographics 

In [7]:
# Remove Duplicate Players From Data Frame to Prevent Over Count of Gender

data_df_dup_rem = data_df.drop_duplicates("SN")


# Gender Demographics DataFrame Filter

females_df = data_df_dup_rem.loc[data_df_dup_rem["Gender"]=="Female",:]
males_df = data_df_dup_rem.loc[data_df_dup_rem["Gender"]=="Male",:]
other_df = data_df_dup_rem.loc[(data_df_dup_rem["Gender"]!="Female")&(data_df["Gender"]!="Male")]

# Gender Demographics Count

num_females = females_df["Gender"].count()
num_males = males_df["Gender"].count()
num_other = other_df["Gender"].count()

# Gender Demographics Percentage

per_females = '{:,.2f}%'.format((num_females/total_players)*100)
per_males = '{:,.2f}%'.format((num_males/total_players)*100)
per_other = '{:,.2f}%'.format((num_other/total_players)*100)


gender_dict = [{"Gender":"Male","Total Count":num_males,"Percentage of Players":per_males},
                {"Gender":"Female","Total Count":num_females,"Percentage of Players":per_females},
                {"Gender":"Other/Non-Disclosed","Total Count":num_other,"Percentage of Players":per_other}
               ]
    
gender_df = pd.DataFrame(gender_dict)
gender_df = gender_df.set_index("Gender")
gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


# Purchase Analysis (Gender)

In [8]:
# Create DataFrame with Total Purchases Per User

data_df1 = data_df.drop_duplicates(subset="SN",keep='first')
data_df1 = data_df1.drop(['Age','Item ID','Item Name','Price'],axis=1)
data_df2 = data_df.groupby(["SN"]).sum()
data_df2 = data_df2.drop(['Age','Item ID'],axis=1)
data_df_merge = pd.merge(data_df1,data_df2,on="SN",how="left")


# Total Players in data_df1 (Same as total_players above)

total_players1 = data_df_merge["SN"].count()

# Split data_df1 (Total Price Spent By All Players) by Gender

TotPrice_females_df = data_df_merge.loc[data_df_merge["Gender"]=="Female"]
TotPrice_males_df = data_df_merge.loc[data_df_merge["Gender"]=="Male"]
TotPrice_other_df = data_df_merge.loc[(data_df_merge["Gender"]!="Female")&(data_df_merge["Gender"]!="Male")]

# Total Spend by Gender 

tot_fem_spend = TotPrice_females_df["Price"].sum()
tot_male_spend = TotPrice_males_df["Price"].sum()
tot_other_spend = TotPrice_other_df["Price"].sum()

# Calculate Average Total Purchase per Person by Gender Group

avg_female_spend = '${:,.2f}'.format(tot_fem_spend/num_females)
avg_male_spend = '${:,.2f}'.format(tot_male_spend/num_males)
avg_other_spend = '${:,.2f}'.format(tot_other_spend/num_other)




In [9]:
# Gender Demographics Data Frame (All Purchases)

all_females_df = data_df.loc[data_df["Gender"]=="Female"]
all_males_df = data_df.loc[data_df["Gender"]=="Male"]
all_other_df = data_df.loc[(data_df["Gender"]!="Female")&(data_df["Gender"]!="Male")]


In [10]:
# Def Funcitions for Calculations

def gender(DFrames):
    gender_title = DFrames.iloc[1,2]
    return gender_title

def gen_count(DFrames):
    tot_purchases = DFrames["SN"].count()
    return tot_purchases

def gen_avg_price(DFrames):
    avg_price = '${:,.2f}'.format(DFrames["Price"].mean())
    return avg_price

def gen_tot_val(DFrames):
    tot_val = '${:,.2f}'.format(DFrames["Price"].sum())
    return tot_val
    
def gen_avg_per(DFrames):
    if DFrames.iloc[2,2]=="Female":
        return avg_female_spend
    elif DFrames.iloc[2,2]=="Male":
        return avg_male_spend
    elif DFrames.iloc[2,2]=="Other / Non-Disclosed":
        return avg_other_spend
    else:
        return "Error"

def gen_dict(DFrames):

    return    {"Gender":gender(DFrames),
               "Purchase Count":gen_count(DFrames),
               "Average Purchase Price":gen_avg_price(DFrames),
               "Total Purchase Valve":gen_tot_val(DFrames),
               "Avg Total Purchase per Person":gen_avg_per(DFrames)
              }

# Generate DataFrame with Def Functions
    
gender_purchase_df = pd.DataFrame([gen_dict(all_females_df),gen_dict(all_males_df),gen_dict(all_other_df)])
gender_purchase_df = gender_purchase_df.set_index("Gender")
gender_purchase_df
    

,Purchase Count,Average Purchase Price,Total Purchase Valve,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


# Age Demographics

In [11]:
# Reset DataFrame CSV Data (Dropped Duplicate Players)

data_df_2nd = pd.read_csv("csv/purchase_data.csv")
data_df_2nd= data_df_2nd.drop_duplicates(["SN"],keep="first")



In [12]:
# Create Bins and Names

bins = [0,9,14,19,24,29,34,39,1000000]
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Cut data_df and Assign New Column Label

data_df_2nd["Age Group"]=pd.cut(data_df_2nd["Age"],bins,labels=group_names,include_lowest=False)


In [13]:
# GroupBy Functions for Display

data_df_grouped = data_df_2nd.groupby(["Age Group"]).count()
data_df_grouped["Percentage of Players"] = ((data_df_grouped["SN"]/(data_df_grouped["SN"].sum()))*100).apply(lambda x: '{:,.2f}%'.format(x))
data_df_grouped = data_df_grouped.drop(['Purchase ID','Age','Gender','Item ID','Item Name','Price'],axis=1)
data_df_grouped = data_df_grouped.rename(columns={"SN":"Total Count"})
data_df_grouped.head(10)


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


# Purchasing Analysis (Age)

In [14]:
# Reset DataFrame CSV Data (Dropped Duplicate Players)

data_df_3rd = pd.read_csv("csv/purchase_data.csv")
data_df_3rd.head()

# Create Bins and Names

bins = [0,9,14,19,24,29,34,39,1000000]
group_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Cut data_df and Assign New Column Label

data_df_3rd["Age Group"]=pd.cut(data_df_3rd["Age"],bins,labels=group_names,include_lowest=False)



In [15]:
# GroupBy Functions for Display
data_grouped2 = data_df_3rd.groupby('Age Group').agg({'Purchase ID':'count',
                                                      'Price':'mean',
                                                      'SN':'nunique', 
                                                     })

data_grouped2 = data_grouped2.rename(columns={'Purchase ID':'Purchase Count',
                                              'Price':'Average Purchase Price'})

data_grouped2['Total Purchase Value'] = data_grouped2['Purchase Count']*data_grouped2['Average Purchase Price']
data_grouped2['Avg Total Purchase per Person'] = data_grouped2['Total Purchase Value']/data_grouped2['SN']
data_grouped2 = data_grouped2.drop(['SN'],axis=1)
pd.options.display.float_format = '${:.2f}'.format
data_grouped2.head(10)



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


# Top Spenders

In [16]:
# Reset DataFrame CSV Data (Dropped Duplicate Players)

data_df_4th = pd.read_csv("csv/purchase_data.csv")


In [17]:
# Group By Function (SN)
data_df_4th = data_df_4th.groupby('SN').agg({'Purchase ID':'count','Price':'mean'})

# Calculate Total Purchase Value
data_df_4th['Total Purchase Value'] = data_df_4th['Purchase ID']*data_df_4th['Price']

# Sort Descending Purchase Count
data_df_4th = data_df_4th.sort_values("Total Purchase Value",ascending=False)

# Rename Columns
data_df_4th = data_df_4th.rename(columns={'Purchase ID':'Purchase Count','Price':'Average Purchase Price'})

data_df_4th.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


# Most Popular Items

In [18]:
# Reset DataFrame CSV Data (Dropped Duplicate Players)

data_df_5th = pd.read_csv("csv/purchase_data.csv")




In [19]:
# Group By Function (Item ID)

data_df_6th = data_df_5th.groupby('Item ID').agg({'Purchase ID':'count'})

data_df_6th = data_df_6th.sort_values('Purchase ID',ascending=False)


In [20]:
# Drop Duplicates Item ID to Merge DataFrame

data_df_7th= data_df_5th.drop_duplicates(["Item ID"],keep="first")
#data_df_7th = data_df_7th.set_index('Item ID')


In [21]:
# Merge data_df_6th & data_df_7th

data_df_8th = pd.merge(data_df_6th,data_df_7th,on="Item ID",how='left')


In [22]:
# Format Column and Add Total Purchase Value

data_df_8th = data_df_8th.drop(['Purchase ID_y','SN','Age','Gender'],axis=1)
data_df_8th = data_df_8th.rename(columns={'Price':'Item Price','Purchase ID_x':'Purchase Count'})
data_df_8th['Total Purchase Value'] = data_df_8th['Purchase Count']*data_df_8th['Item Price']
data_df_8th = data_df_8th[['Item ID','Item Name','Purchase Count','Item Price','Total Purchase Value']]
data_df_8th = data_df_8th.set_index('Item ID')
data_df_9th = data_df_8th.sort_values('Purchase Count',ascending=False)
data_df_9th.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.88,$63.44
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.71
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


# Most Profitable Items

In [23]:
# Call DataFrame (data_df_8th) and Sort Values Decsending Total Purchase Value

data_df_10th = data_df_8th.sort_values('Total Purchase Value',ascending=False)
data_df_10th.head(5)

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.88,$63.44
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
